<a href="https://colab.research.google.com/github/nachiketaraina/ndvi_using_gee/blob/master/test_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
!pip install ipygee #ipygee is a tool helpful in map

     |████████████████████████████████| 2.8MB 10.2MB/s 
     |████████████████████████████████| 133kB 57.9MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
     |████████████████████████████████| 225kB 49.9MB/s 
  Created wheel for ipygee: filename=ipygee-0.0.14-cp36-none-any.whl size=39897 sha256=35c5db988aff954e90601460b6a473a1b11dee43302cd102074dc2cd48e8a2df
  Stored in directory: /root/.cache/pip/wheels/57/f1/a2/ce9509a072ede592c0c9239ba66a1e9bb8af37bab3ca72fabc
  Created wheel for geetools: filename=geetools-0.4.15-cp36-none-any.whl size=98234 sha256=9d91c96e087643513d4e4e0e0aa023bb476cc4c8440785a12f375e5f86650b1c
  Stored in directory: /root/.cache/pip/wheels/a9/29/4b/8b23d7a814fd5b03acab81a0a721c4dc0d782fed28760b636a
  Created wheel for pyshp: filename=pyshp-2.1.0-cp36-none-any.whl size=32609 sha256=5f086cd3f66fe9a82dc0ff67f3e927d5ae3b817d0f3defd0b9448780624de82b
  Stored in directory: /root/.cache/pip/wheels/a6/0c/de/321b5192ad416b328975a2f0385f72c64db4656501eba7cc1a

In [0]:
import ee
import time
from ipygee import *
#import  // ui ui has not been made available in python development enviroment as of yet
from IPython.display import Image
import folium
#adding ee_layer to folium to display maps
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer



In [3]:
!earthengine authenticate

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

/usr/local/lib/python3.6/dist-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=mkjCSVh9BBKW5NCMY3dFxzjvBjP249bXOGMmMLl2J4w&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

from google.colab import auth
auth.authenticate_user()

In [0]:
a1 = ee.Feature(ee.Geometry.Point(13.8463669499,52.9008472902),{'label':'a1'})#selecting a random region
a2 = ee.Feature(ee.Geometry.Point(13.778028117,52.951728864),{'label ':'a1'})

In [0]:
cord = ee.FeatureCollection([a1,a2]) #making a feature collection

In [0]:
ee.Initialize() 

In [0]:
#function for computing nvdi from red
def NDVI(image):
  ndvi =image.normalizedDifference(['B4','B3']).rename('NDVI')
  return image.addBands(ndvi)


In [0]:
#function for computing nvdi from red and near infrared bands

def addNDVI2(image):
  ndvi = image.normalizedDifference(['B5','B4']).rename('NDVI2')
  return image.addBands(ndvi);


In [0]:
def addFmask(image):
  datamask = image.select('fmask')
  cloudMask = datamask.neq(1)and(datamask.neq(2))and(datamask.neq(3))and(datamask.neq(4))
  return image.updateMask(cloudMask)

In [0]:
image_collection = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")# ('Quearing imageset for landsat-8 images')

In [0]:
time_series = image_collection.filterDate('2016-01-01','2018-12-30') #time_series over 2 years 

In [0]:
 #selecting netherland as random location
pathrow=time_series.filterBounds(cord)

In [0]:
clouds = pathrow.filter(ee.Filter.lt('CLOUD_COVER', 5))   #selecting images having less cloud cover than 5 percent

In [12]:
count = clouds.size()
print('Count: ', str(count.getInfo())+'\n')#size of collection of images

Count:  18



In [0]:
bands = clouds.select(['B4','B5'])

In [0]:
resultant_ndvi = bands.map(addNDVI2)

In [0]:
# custom export function as gee does not provide so
collectionList = resultant_ndvi.toList(resultant_ndvi.size())
collectionSize = collectionList.size().getInfo()
for i in range(collectionSize):
    ee.batch.Export.image.toDrive(
        image = ee.Image(collectionList.get(i)), 
        fileNamePrefix = 'foo' + str(i + 1), 
        dimensions = '128x128').start()

Below is some research work that i am looking on internet to find support for python api that is not exclusively supported

In [30]:
composite = resultant_ndvi.qualityMosaic('NDVI').clip(cord);
print(composite);

ndviPalette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301'];
Map.addLayer(composite.select('NDVI'),
            {min:0, max: 1,' palette': ndviPalette});

ee.Image({
  "type": "Invocation",
  "arguments": {
    "input": {
      "type": "Invocation",
      "arguments": {
        "collection": {
          "type": "Invocation",
          "arguments": {
            "collection": {
              "type": "Invocation",
              "arguments": {
                "collection": {
                  "type": "Invocation",
                  "arguments": {
                    "collection": {
                      "type": "Invocation",
                      "arguments": {
                        "collection": {
                          "type": "Invocation",
                          "arguments": {
                            "collection": {
                              "type": "Invocation",
                              "arguments": {
                                "collection": {
                                  "type": "Invocation",
                                  "arguments": {
                                    "id": "LANDSAT/LC08/C01/T1_SR

EEException: ignored

In [16]:

dem = ee.Image('USGS/SRTMGL1_003')

Image(url = clouds.median().updateMask(clouds.median())
  .getThumbUrl({'min': 0, 'max': 2000, 'dimensions': 512,
             }))

In [0]:
import google

ndviParams = {min: -1, max: 1}
my_map = folium.Map(location = [52,5],height=500,zoom_start  = 3)
my_map.add_ee_layer(im.updateMask(im), ndviParams, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [0]:
folium.Map.add_ee_layer()

In [0]:
pip install paver

In [0]:
!paver.tasks.help


/bin/bash: paver.tasks.help: command not found


In [0]:
image_list = clouds.toList(clouds.size())

In [0]:
while out.active():
  print('Polling for task (id: {}).'.format(out.id))
  time.sleep(5)

Polling for task (id: WUYX4K6V5GVUBGR43R4KNH2V).
Polling for task (id: WUYX4K6V5GVUBGR43R4KNH2V).
Polling for task (id: WUYX4K6V5GVUBGR43R4KNH2V).


In [0]:
out.start()

In [0]:
geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])

In [0]:
out  = ee.batch.Export.image.toDrive(clouds,folder = 'aa')

In [0]:
out = ee.batch.Export.image.toDrive(image = band_image,folder = 'aa',maxPixels=3784216672400,region=roi)

In [0]:
out.status()

{'creation_timestamp_ms': 1584543777937,
 'description': 'myExportImageTask',
 'destination_uris': ['https://drive.google.com/#folders/1gIa7nL2cHDdVM4jZeM0vcrHhrNCVSMUF'],
 'id': 'EV2NC43UHOCKMJ7U42HTCKQA',
 'name': 'projects/earthengine-legacy/operations/EV2NC43UHOCKMJ7U42HTCKQA',
 'start_timestamp_ms': 1584543787901,
 'state': 'COMPLETED',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1584544379693}

In [0]:
im = clouds.first()

In [0]:
roi=ee.Feature(clouds.first()).geometry().bounds().getInfo()['coordinates']

In [0]:
m = folium.Map(clouds,zoom_start=15)

TypeError: ignored

In [0]:
im  =  clouds.first()

In [0]:
n = im.normalizedDifference(['B4','B5'])
band_image = im.add(n)

In [34]:
display(band_image)

In [0]:
Image(,format='jpg')

In [38]:
NDVI(im)

In [0]:
import ee.mapclient

In [0]:
map = ee.mapclient.centerMap(52,5,6)

In [0]:
imgndvi = clouds.first().normalizedDifference(['B5', 'B4']).rename('NDVI')

In [0]:
ndviParams = {min: -1, max: 1, 'palette': ['blue', 'white', 'green']};
Map.addLayer(imgndvi, ndviParams);

In [28]:
dislay(map)

NameError: ignored

In [0]:
Map = Map()

In [0]:
Map.addLayer